In [1]:
import pandas as pd
import numpy as np

dtypes ={
    'id': np.uint32,
    'qid1': np.uint32,
    'qid2': np.uint32,
    'question1': np.str,
    'question2': np.str,
    'is_duplicate': np.uint8
}

df = pd.read_csv('../dataset/quora_train.csv', dtype=dtypes)
df_test = pd.read_csv('../dataset/quora_train.csv', dtype=dtypes, iterator=True, chunksize=10000)

In [2]:
df.describe()

,id,qid1,qid2,is_duplicate
count,404290.000000,404290.000000,404290.000000,404290.000000
mean,202144.500000,217243.942418,220955.655337,0.369198
std,116708.614503,157751.700002,159903.182629,0.482588
min,0.000000,1.000000,2.000000,0.000000
25%,101072.250000,74437.500000,74727.000000,0.000000
50%,202144.500000,192182.000000,197052.000000,0.000000
75%,303216.750000,346573.500000,354692.500000,1.000000
max,404289.000000,537932.000000,537933.000000,1.000000


In [3]:
import spacy

nlp = spacy.load('en')

In [4]:
def compare(q1, q2):
    q1 = q1 if type(q1) is str else ''
    q2 = q2 if type(q2) is str else ''
    
    qt1 = [t for t in nlp(q1) if t.tag_.startswith('N')]# or t.tag_.startswith('V') or t.tag_.startswith('J')]
    qt2 = [t for t in nlp(q2) if t.tag_.startswith('N')]# or t.tag_.startswith('V') or t.tag_.startswith('J')]
    
    count = 0
    s_sum = 0
    for t1 in qt1:
        for t2 in qt2:
            if t1.tag_[0] == t2.tag_[0]:
                count += 1
                s_sum += t1.similarity(t2)
    
    return s_sum / count if count != 0 else 0

In [5]:
from multiprocessing import Pool

pool = Pool(4)

df['compare'] = pool.starmap(compare, tuple(zip(df.question1, df.question2)))

Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/ic/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/ic/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ic/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/Users/ic/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/ic/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
Process ForkPoolWorker-1:
  File "/Users/ic/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    s

In [8]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(df.compare.values.reshape(-1, 1), df.is_duplicate.values)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
from tqdm import tqdm

output = open('result.csv', 'w')
output.write('test_id,is_duplicate')

for chunk in tqdm(df_test):
    chunk['compare'] = pool.starmap(compare, tuple(zip(chunk.question1, chunk.question2)))
    prediction = clf.predict_proba(chunk['compare'].reshape(-1, 1))
    
    for i, p in zip(chunk['id'], prediction):
        output.write('{},{}'.format(i, p[1]))

0it [00:00, ?it/s]/Users/ic/.pyenv/versions/3.6.1/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
39it [05:10,  7.21s/it]
